In [1]:
import torchvision
import torch
import torch.nn as nn
import sys
sys.path.append("../misc");
from utils import DataShaper
import os
import math
import matplotlib.pyplot as plt
import numpy as np
from brevitas.nn import QuantConv2d, QuantIdentity, QuantReLU
from brevitas.quant.fixed_point import Int8ActPerTensorFixedPoint, Int8WeightPerTensorFixedPoint,Uint8ActPerTensorFixedPoint

torch.manual_seed(0)
design="bottleneck_init_cifar_split_scalar"
# aie_teardown()
sys.path.append("../../../utils"); import xrtutils
xclbin_path = os.path.abspath("../bottleneck_block/"+design+"/final.xclbin")
insts_path  = os.path.abspath("../bottleneck_block/"+design+"/insts.txt")

log_folder="log/log_"+design

enable_aie = True
aie_is_setup = False
enable_trace = False
trace_file='traces/trace.txt'

app = None
in_buf = None
arg1_buf = None
out_buf = None
dtype_in  = np.dtype("int8")
dtype_wts  = np.dtype("int8")
dtype_out = np.dtype("uint8")

shape_in_act   = (32,8,32,8)  #'YCXC8' , 'CYX'
shape_in_wts1  = (8,8,1,1,8,8) #out,in,ky,kx,in8,out8
shape_in_wts2  = (8,8,3,3,8,8)  #out,in,ky,kx,in8,out8
shape_in_wts3  = (32,8,1,1,8,8)   #out,in,ky,kx,in8,out8
shape_in_wts_skip  = (8,32,1,1,8,8) #out,in,ky,kx,in8,out8
shape_total_wts= (73728,1)
shape_out      = (32,32,32,8)

trace_size = 8192

def setup_aie(xclbin_path, insts_path, 
              in_0_shape, in_0_dtype,
              in_1_shape, in_1_dtype, 
              out_buf_shape, out_buf_dtype,
              enable_trace=False,
              kernel_name="MLIR_AIE"):
    app = xrtutils.AIE_Application(xclbin_path, insts_path, kernel_name)
    app.register_buffer(2, shape=in_0_shape, dtype=in_0_dtype)
    app.register_buffer(3, shape=in_1_shape, dtype=in_1_dtype)
    if enable_trace:
      out_buf_len_bytes = np.prod(out_buf_shape) * np.dtype(out_buf_dtype).itemsize
      out_buf_shape = (out_buf_len_bytes + trace_size, )
      out_buf_dtype = np.uint8
    app.register_buffer(4, shape=out_buf_shape, dtype=out_buf_dtype)
    return app

def extract_trace(out_buf, out_buf_shape, out_buf_dtype):
    trace_size_words = trace_size//4
    out_buf_flat = out_buf.reshape((-1,)).view(np.uint32)
    output_prefix = out_buf_flat[:-trace_size_words].view(out_buf_dtype).reshape(out_buf_shape)
    trace_suffix = out_buf_flat[-trace_size_words:]
    return output_prefix, trace_suffix

def write_out_trace(trace, file_name):
    out_str = "\n".join(f"{i:0{8}x}" 
                        for i in trace
                        if i != 0)
    with open(file_name, 'w') as f:
      f.write(out_str)

app = setup_aie(xclbin_path, insts_path,
                            shape_in_act, dtype_in,      
                            shape_total_wts,dtype_wts,
                            shape_out, dtype_out,enable_trace)

import matplotlib.pyplot as plt
import numpy as np
from brevitas.nn import QuantConv2d, QuantIdentity, QuantReLU
from brevitas.quant.fixed_point import Int8ActPerTensorFixedPoint, Int8WeightPerTensorFixedPoint,Uint8ActPerTensorFixedPoint

torch.manual_seed(0)
torch.use_deterministic_algorithms(True)

if not os.path.exists(log_folder):
    os.makedirs(log_folder)

input=torch.randn(1, 64,32,32)
num_classes=10
ds = DataShaper()
def init_pad_input(x,input_channels,desired_channels=4):
           padding=torch.zeros(1,input_channels*(desired_channels-1),32,32)
           return torch.cat((x,padding),1)
# try:
for i in range (0,1): 
    class QuantBottleneck_projected(nn.Module):
        expansion = 4
        def __init__(self, in_planes=64, planes=64):
            super(QuantBottleneck_projected, self).__init__()
            self.quant_id_1 = QuantIdentity(act_quant=Int8ActPerTensorFixedPoint,bit_width=8, return_quant_tensor=True) 

            self.quant_conv1 = QuantConv2d(in_planes, planes, kernel_size=1,bit_width=8,weight_bit_width=8, bias=False, weight_quant=Int8WeightPerTensorFixedPoint, return_quant_tensor=True)
            self.quant_conv2 = QuantConv2d(planes, planes, kernel_size=3,bit_width=8,weight_bit_width=8, bias=False,padding=1,padding_mode ='zeros', weight_quant=Int8WeightPerTensorFixedPoint, return_quant_tensor=True)
            self.quant_conv3 = QuantConv2d(planes, self.expansion *planes, kernel_size=1,bit_width=8,weight_bit_width=8, bias=False,weight_quant=Int8WeightPerTensorFixedPoint, return_quant_tensor=True)
            self.quant_relu1 = QuantReLU(act_quant=Uint8ActPerTensorFixedPoint ,bit_width=8, return_quant_tensor=True)
            self.quant_relu2 = QuantReLU(act_quant=Uint8ActPerTensorFixedPoint ,bit_width=8, return_quant_tensor=True)
            self.quant_relu3 = QuantReLU(act_quant=Uint8ActPerTensorFixedPoint ,bit_width=8, return_quant_tensor=True)
            self.shortcut = QuantConv2d(in_planes, self.expansion*planes, kernel_size=1,bit_width=8,weight_bit_width=8, bias=False, weight_quant=Int8WeightPerTensorFixedPoint, return_quant_tensor=True)
                
        def forward(self, x):
            out_q = self.quant_id_1(x)
            
            print("Before:::",out_q.size())
            # print(out_q[0][0][63])
            out = self.quant_conv1(out_q)
            out = self.quant_relu1(out)
            out = self.quant_conv2(out)
            out = self.quant_relu2(out)
            out = self.quant_conv3(out)
            out = self.quant_id_1(out)
            out_lhs=self.shortcut(out_q)
            out_lhs = self.quant_id_1(out_lhs)
            out = out+out_lhs
            out = self.quant_relu3(out)
            return out
            

    
    quant_bottleneck_model=QuantBottleneck_projected()
    
    quant_id_1 = QuantIdentity(act_quant=Int8ActPerTensorFixedPoint,bit_width=8, return_quant_tensor=True) 
    quant_bottleneck_model.eval()
    quant_id_1.eval()
  
    init_scale = quant_bottleneck_model.quant_id_1.quant_act_scale()
    block_0_relu_1 = quant_bottleneck_model.quant_relu1.quant_act_scale()
    block_0_relu_2= quant_bottleneck_model.quant_relu2.quant_act_scale()
    block_0_relu_3= quant_bottleneck_model.quant_relu3.quant_act_scale()
    block_0_weight_scale1 = quant_bottleneck_model.quant_conv1.quant_weight_scale()
    block_0_weight_scale2 = quant_bottleneck_model.quant_conv2.quant_weight_scale()
    block_0_weight_scale3 = quant_bottleneck_model.quant_conv3.quant_weight_scale()
    block_0_weight_scale_skip = quant_bottleneck_model.shortcut.quant_weight_scale()
    
    block_0_int_weight_1 = quant_bottleneck_model.quant_conv1.quant_weight().int(float_datatype=True)
    block_0_int_weight_2 = quant_bottleneck_model.quant_conv2.quant_weight().int(float_datatype=True)
    block_0_int_weight_3 = quant_bottleneck_model.quant_conv3.quant_weight().int(float_datatype=True)
    block_0_int_weight_skip = quant_bottleneck_model.shortcut.quant_weight().int(float_datatype=True)
    
    block_0_combined_scale1=-torch.log2(init_scale*block_0_weight_scale1/block_0_relu_1)
    block_0_combined_scale2=-torch.log2(block_0_relu_1*block_0_weight_scale2/block_0_relu_2)
    block_0_combined_scale3=-torch.log2(block_0_relu_2*block_0_weight_scale3/init_scale)
    block_0_combined_scale4=-torch.log2(init_scale/block_0_relu_3)

    block_0_combined_scale_skip=-torch.log2(init_scale*block_0_weight_scale_skip/init_scale)

    print("combined_scale after first conv1x1:",block_0_combined_scale1.item())
    print("combined_scale after second conv3x3:",block_0_combined_scale2.item())
    print("combined_scale after third conv1x1:",block_0_combined_scale3.item())
    print("combined_scale after adding skip connection:",(block_0_combined_scale4).item())
    print("combined_scale after skip conv1x1:",block_0_combined_scale_skip.item())

    q_bottleneck_out=quant_bottleneck_model(input)
    gold_out=q_bottleneck_out.int(float_datatype=True).data.numpy().astype(dtype_out)
    print("Golden::Brevitas::",gold_out)
    from brevitas.export import export_onnx_qcdq
    # ref_input = torch.ones(1, 3, 32, 32, device="cpu", dtype=dtype)
    export_onnx_qcdq(quant_bottleneck_model, input, log_folder+"/"+design+".onnx")
    # # Brevitas convolution
    q_inp = quant_id_1(input)
    int_inp = q_inp.int(float_datatype=True)
    
    before_input=int_inp.squeeze().data.numpy().astype(dtype_in)

    before_input.tofile(log_folder+"/before_ifm_mem_fmt_1x1.txt", sep=",", format="%d")
    ifm_mem_fmt = ds.reorder_mat(before_input,'YCXC8' , 'CYX' )
    ifm_mem_fmt.tofile(log_folder+"/after_ifm_mem_fmt_1x1.txt", sep=",", format="%d")
    wts1 = ds.reorder_mat(block_0_int_weight_1.data.numpy().astype(dtype_wts),'OIYXI8O8' , 'OIYX' )
    wts2 = ds.reorder_mat(block_0_int_weight_2.data.numpy().astype(dtype_wts),'OIYXI8O8' , 'OIYX' )
    wts3 = ds.reorder_mat(block_0_int_weight_3.data.numpy().astype(dtype_wts),'OIYXI8O8' , 'OIYX' )
    wts_skip = ds.reorder_mat(block_0_int_weight_skip.data.numpy().astype(dtype_wts),'OIYXI8O8' , 'OIYX' )

    total_wts=np.concatenate((wts1,wts2,wts3,wts_skip),axis=None)
    total_wts.tofile(log_folder+"/weights_mem_fmt_final.txt", sep=",", format="%d")
    print("total_wts", total_wts.shape)
    for i in range (0,2):
        app.buffers[2].write(ifm_mem_fmt)# input's standard format CYX | scalar YCX
        app.buffers[3].write(total_wts) # wts's standard format OIYX | scalar OIYX
        # app.buffers[3].write(int_weight2.data.numpy().astype(dtype_in),offset=2048) # wts's standard format OIYX | scalar OIYX
        app.run()
        output3= app.buffers[4].read()
        if enable_trace:
            output3, trace = extract_trace(output3, shape_out, dtype_out)
            write_out_trace(trace, trace_file)
    # temp_out=output3.reshape(32,256, 32)
    # ofm_mem_fmt = temp_out.swapaxes(0,1)
    temp_out    = output3.reshape(32,32,32,8)
    temp2_out   = ds.reorder_mat( temp_out, 'CDYX','YCXD' )
    ofm_mem_fmt = temp2_out.reshape(256,32,32)   
    ofm_mem_fmt.tofile(log_folder+"/after_ofm_mem_fmt_final.txt", sep=",", format="%d")

    ofm_mem_fmt=torch.from_numpy(ofm_mem_fmt).unsqueeze(0)
    print("AIE output:::",ofm_mem_fmt)
    print(type(ofm_mem_fmt))
    print(type(q_bottleneck_out))
    print("difference::",torch.max(torch.abs(ofm_mem_fmt*block_0_relu_3 - q_bottleneck_out)))
    assert(np.allclose(ofm_mem_fmt, gold_out, rtol=0, atol=2.))



combined_scale after first conv1x1: 9.0
combined_scale after second conv3x3: 11.0
combined_scale after third conv1x1: 11.0
combined_scale after adding skip connection: -1.0
combined_scale after skip conv1x1: 10.0
Before::: torch.Size([1, 64, 32, 32])
Golden::Brevitas:: [[[[  0   0 144 ...   0   0   0]
   [  0   0   0 ...   4   0   0]
   [250   0   0 ...   0 122  70]
   ...
   [  0  64   0 ...   0  10 120]
   [  0  46  96 ... 208  76 120]
   [  0 110  22 ... 114   0   0]]

  [[  0   0 174 ...  84   0  24]
   [ 66 114   0 ...  66   0   0]
   [128   0   0 ...   0   6   0]
   ...
   [ 82  32  42 ...   2  84 198]
   [  0  10  48 ...   0   0   0]
   [  0  44  50 ...   0   0  52]]

  [[  0 136   0 ...   0 230 118]
   [ 36   0   0 ...   8 126  56]
   [  6   4 200 ...  26 255  18]
   ...
   [  0   0   0 ...  30   0  16]
   [ 40   0   0 ...   0   0   0]
   [ 98   0 118 ...   0  40   0]]

  ...

  [[ 22  46   0 ...  76   0   6]
   [ 50   0   0 ... 204   0   0]
   [222  20  48 ...   0   0   0]
   

C:\Users\gagan\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\_tensor.py:1271: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/core/TensorImpl.h:1791.)
  return super().rename(names)
C:\Users\gagan\AppData\Local\Programs\Python\Python39\lib\site-packages\brevitas\export\onnx\standard\manager.py:26: UserWarning: ONNX opset version set to 13, override with opset_version=
  warnings.warn(f"ONNX opset version set to {DEFAULT_OPSET}, override with {ka}=")


============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

total_wts (73728,)
AIE output::: tensor([[[[  0,   0, 144,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   4,   0,   0],
          [250,   0,   0,  ...,   0, 122,  70],
          ...,
          [  0,  64,   0,  ...,   0,  10, 120],
          [  0,  46,  96,  ..., 208,  76, 120],
          [  0, 110,  22,  ..., 114,   0,   0]],

         [[  0,   0, 174,  ...,  84,   0,  24],
          [ 66, 114,   0,  ...,  66,   0,   0],
          [128,   0,   0,  ...,   0,   6,   0],
          ...,
          [ 82,  32,  42,  ...,   2,  84, 198],
          [  0,  10,  48,  ...,   0,   0,   0],
          [  0,  44,  50,  ...,   0,   0,  52]],

         [[  0, 136,   0,  ...,   0, 230, 118],
          [ 36,   0,   0,  ...,   8, 126,  56],
          [  6,   4, 200,  ...,  26, 255,  18],
          .

C:\Users\gagan\AppData\Local\Temp\ipykernel_15220\248015336.py:208: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\python_arg_parser.cpp:367.)
  print("difference::",torch.max(torch.abs(ofm_mem_fmt*block_0_relu_3 - q_bottleneck_out)))


In [ ]:
if (enable_trace):
    print(trace)
else:
    print ("tracing not enabled")